# BERT模型
BERT模型是利用Transformer结构通过完形填空语料库的自监督学习方法来进行预训练的模型。Bert的特点在于，作为一种预训练模型，它编码的上下文信息已经十分的丰富，以至于在这种预训练模型上只需要微调（fine tuning）即可在下游任务上达到非常好的效果。

BERT的代码模型示意图如下：绘制该图时主要参考的是Huggingface的Transformers源码：

<img src="https://image.panwenbo.icu/blog20210729164102.png" alt="截屏2021-07-29 下午4.36.25" style="zoom:40%;" />

接下来简单实现一个基础的BERT模型。首先来实现Bert的Embedding层，Bert的Embedding由三种Embedding相加而成，词嵌入，位置编码和Token类型编码。其中值得注意的是位置编码不使用三角函数编码而是把词语的位置传入一个可以学习的Embedding层学习得到；Token类型编码用于帮助Bert区分输入中的两句不同的话（问题——答案，前因——后果等），它是把0或者1代入一个可学习的Embedding层得到的。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

config = {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

class BertEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding = nn.Embedding(config["vocab_size"], config["hidden_size"])
        self.positional_embedding = nn.Embedding(config["max_position_embeddings"], config["hidden_size"])
        self.token_type_embedding = nn.Embedding(config["type_vocab_size"], config["hidden_size"])
        self.layernorm = nn.LayerNorm(config["hidden_size"])
        self.dropout = nn.Dropout(config["hidden_dropout_prob"])

    def forward(self, input_ids, pos_ids, token_type_ids):
        embed = (self.token_embedding(input_ids)
                 + self.positional_embedding(pos_ids)
                 + self.token_type_embedding(token_type_ids))
        return self.dropout(self.layernorm(embed))

接下来我们直接实现一整个Bert。下面我们直接使用nn.TranformerEncoderLayer来实现Bert的BertLayer。实际上Huggingface实现的Bert中的BertAttention在就对应于Transformer中的Multi-Head Attention，而Intermediate和Output合起来就是我们常说的FFN。这个Bert的forward函数返回Encoder的隐藏层输出（除[CLS]）和[CLS]字符对应的pool输出。

In [9]:
class Bert(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = BertEmbedding(config)

        bert_layer = nn.TransformerEncoderLayer(
            d_model=config["hidden_size"], 
            nhead=config["num_attention_heads"],
            dim_feedforward=config["intermediate_size"],
            dropout=config["attention_probs_dropout_prob"],
            activation=config["hidden_act"]
            )
        self.encoder = nn.TransformerEncoder(bert_layer, config["num_hidden_layers"])

        self.pooler = nn.Sequential(
            nn.Linear(config["hidden_size"], config["hidden_size"]),
            nn.Tanh()
            )
    
    def forward(self, src, src_token_type, src_mask, src_padding_mask):
        src_len, batch_size = src.shape
        src_pos_ids = torch.arange(src_len).reshape(src_len, 1).expand(src_len, batch_size)
        embed = self.embedding.forward(src, src_pos_ids, src_token_type)

        hiddens = self.encoder(embed, src_mask, src_padding_mask)

        # With hiddens shape:(Length, Batch_size, d_model), pooler computes
        # the first token of source sequence([CLS])
        pooled = self.pooler(hiddens[0])
        return hiddens[1:], pooled

最后我们再定义一下Bert的两种训练头，MLMHead和NSPHead。但是全部完成以后，我们不会真正地训练Bert，因为Bert的训练肯定是需要GPU的，同时花费的时间比较长，其中需要的技巧也要求比较复杂的代码。其实更好的方法是了解一下Huggingface的Transformers库中的预训练Bert和它的几种变体。

In [10]:
class BertMLMHead(nn.Module):
    def __init__(self, bert_model: Bert, config):
        super().__init__()
        self.dense = nn.Linear(config["hidden_size"], config["hidden_size"])
        self.layernorm = nn.LayerNorm(config["hidden_size"])
        self.decoder = nn.Linear(config["hidden_size"], config["vocab_size"])
        self.decoder.weight = bert_model.embedding.token_embedding.weight
    
    def forward(self, hiddens):
        hiddens = self.dense(hiddens)
        hiddens = F.gelu(hiddens)
        hiddens = self.layernorm(hiddens)
        return self.decoder(hiddens)
    

class BertNSPHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear = nn.Linear(config["hidden_size"], 2)
    
    def forward(self, pooled):
        return self.linear(pooled)

使用Huggingface/transformers库中的AutoTokenizer来编码几个输入，输入Bert看看能否正常输出。检查一下Bert的总参数量，基本上和原始实现中的规模是一致的。

In [13]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
text = ["How is everything [MASK] today?",
        "Tranformer models are advance models; they [MASK] helpful in nlp task."]

# Load tokenizer of model "bert-base-uncased"
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokens = tokenizer(text, padding=True)

# There ars three items in tokens: input_ids, token_type_ids and attention_mask, all in form of List[List[int]].
# And all three variable are arranged as batch-first.
# However, in pytorch's implement, attention_mask is a ByteTensor, where True means masked, False means unmasked
# while in Tranformers, 1 means unmasked and 0 means masked. And pytorch use length-first as default, so some 
# changes are needed.
input_ids = torch.tensor(tokens["input_ids"]).long().T
token_type_ids = torch.tensor(tokens["token_type_ids"]).long().T
attention_mask = torch.tensor(tokens["attention_mask"]) == 0

print("Running Bert model...")
model = Bert(config)
mlm_head = BertMLMHead(model, config)
nsp_head = BertNSPHead(config)

hiddens, pooled = model(input_ids, token_type_ids, None, attention_mask)
tok_probs = mlm_head(hiddens)
cls_probs = nsp_head(pooled)

print("Total param numbers of Bert: ", sum([p.numel() for p in model.parameters()]))
print("MLM Prediction shape: ", tok_probs.shape)
print("NSP Prediction shape: ", cls_probs.shape)

Loading tokenizer...
Running Bert model...
Total param numbers of Bert:  109482240
MLM Prediction shape:  torch.Size([17, 2, 30522])
NSP Prediction shape:  torch.Size([2, 2])


## Huggingface/Transformers
接下来我们介绍一下Transformers库中的几种Bert变体（严格来说是Transformer的预训练模型应用）。所有的Transfomer的预训练模型本质上都是语言模型。（以下内容大量参考自Huggingface官网教程）
Tranformer作为一个Encoder-Decoder的结构，它的Encoder，Decoder，Encoder-Decoder结构都可以单独成为预训练模型，因此大体上可以把近几年的预训练模型分为以下三类：

1. Bert-like（也叫做自编码Transformer模型）

    自编码Transformer模型只选用了Transformer中的Encoder结构，拥有一一对应且等长等大小的输入和输出序列。它们一般通过遮罩语言模型的方法训练（遮住某几个词语并要求模型预测），尤其擅长理解一整句话并进行标注，分类等任务。代表模型有Bert，AlBert，RoBerta，DistilBert和ELECTRA，中文的还有ERNIE。

2. GPT-like（也叫做自回归Transformer模型）

    自回归Transformer模型则只是Transformer中的Decoder结构，运行时需要根据一开始的一小段文字，在这批文字的基础上得到最可能的下一个词，并把这个词加入最开始的一小段文字，重复迭代不断加长文字。这些结构尤其擅长生成语言的任务。代表模型有CTRL，GPT，GPT-2/3，Tranformer-XL等。

3. BART/T5-like（也叫做序列到序列Transformer模型）

    序列到序列Transformer模型就是一整个的Encoder-Decoder，可以接受输入和产生输出，并且输出输入之间没有长度关系要求也没有一一对应关系。它们的预训练方法更加复杂，在微调阶段则适用于根据已有文本生成新文本的任务，如翻译，总结或生成回答等等。

![transformers_chrono](https://image.panwenbo.icu/blog20210730102030.png)

图片来自于Huggingface教程https://huggingface.co/course


### Roberta
Roberta（Robustly Optimized BERT Pretraining Approach）是由Facebook AI提出的改善Bert模型的方法，在论文《RoBERTa: A Robustly Optimized BERT Pretraining Approach》中被提出。Roberta在模型结构上和Bert基本上没有差别，除了删除了NSP任务导致没有Token_type_embedding以外和Bert没有任何区别。它的贡献主要体现于：
1. 增大了训练数据量（从16G增加到了160G），增大了batch_size（增加到8000）
2. 使用动态Mask代替了Bert的静态Mask，在每次向模型输入数据之前随机的选择15%的单词进行mask，而不是Bert在建立数据集一开始就选择了哪些词被masked。
3. 删除了Next Sentence Prediction任务。实验发现在512的长度内加入一整句话能够比加入两段比较短的语句并进行NSP能够达到更好的成绩。而且，在有NSP的情况下，如果这两句话是长句子中切割的两部分又能够比相互独立的两句话得到更好的训练效果。可见句子的长度可能是很重要的。模型需要长句子来学习远程依赖。而本身NSP难度又过低，可能起不到什么监督作用，因此Roberta直接删除了NSP任务。
4. 词嵌入阶段使用BPE编码而不是WordPiece编码，引入了更多的参数，增大了模型的容量。


In [32]:
from transformers import RobertaTokenizer, BertTokenizer

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokens = roberta_tokenizer.tokenize("Hello my name is thisisname")
print("BPE Tokens: ", tokens)

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokens = bert_tokenizer.tokenize("Hello my name is thisisname")
print("WordPiece Tokens: ", tokens)

BPE Tokens:  ['Hello', 'Ġmy', 'Ġname', 'Ġis', 'Ġthis', 'is', 'name']
WordPiece Tokens:  ['hello', 'my', 'name', 'is', 'this', '##is', '##name']


### DistilBert

DistilBert就是Bert的知识蒸馏版本。DistilBert去掉了Bert的Pooler输出和Token_type_embedding，并减少层数至一半，使用从Bert中每两层抽取一层的参数的方法来进行初始化。之后DistilBert学习Bert的输出分布，在保留了97%的精度的同时还减少40%的大小，快了60%。

但是从论文的数据来看，DistilBert的精度损失还是比较严重的，毕竟大家都扣着那一两个点竞争，损失个3%也不少了。毕竟，下面将要提到的Albert-base也可以说是在减少了89%的大小情况下保留了97%的精度。

In [33]:
# 输出一下可以看到DistilBert只有6层。
from transformers import DistilBertConfig
print(DistilBertConfig())

DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.9.1",
  "vocab_size": 30522
}



### ELECTRA
ELECTR是Google在《ELECTRA: Pre-Training Text Encoders as Discriminators rather than generators》中提出的模型，它的核心目的是为了解决Bert由于预训练时有`[MASK]`而微调时没有`[MASK]`造成的mismatch问题。ELECTRA分为两个部分———generator和discriminator模型，generator一般是一个小一点的MLM语言模型，负责把输入文本里的`[MASK]`预测成最可能的单词，但是generator可能犯错，有可能它预测的部分词语并不是文本中本来的词语，于是我们的Discriminator的任务就是把这些预测错误的词语找出来。因此在Discriminator眼里，它得到的句子都是看起来很正常的句子，它要解决的也是个一一对应的二分类问题，把Bert模型应用在这上面就可以得到一个不会有Mismatch问题的Bert。

![transformers_chrono](https://image.panwenbo.icu/blog20210730160709.jpg)
实验显示，Generator应该是一个略小于Discriminator的Bert，Generator既不能太弱导致无法给Discriminator施加压力，也不能太强导致Discriminator难以学习。同时，让两者同时训练，相对于分先后顺序的训练，可以使ELECTRA最快收敛。ELECTRA在原文中训练了三种规模ELECTRA，ELECTRA-small和ELECTRA-large，和Bert大中小都有着一致的参数，但是均可以达到明显更优的表现。


看到ELECTRA，很多人第一时间就会想到GAN，但是其实相对于GAN，作者自己也提到ELECTRA其实更像一个使用负采样的CBOW词嵌入算法，毕竟它们都是根据上下文确定目标词语是否属于该语境。


### AlBert
AlBert（A LITE BERT FOR SELF-SUPERVISED LEARNING OF LANGUAGE REPRESENTATIONS）是由谷歌提出来的另一种Bert的改进版本，相比于RoBerta对于Bert原始模型未做丝毫改动，Albert对模型做了以下几种改进：

1. 减小WordEmbedding的大小

理论上，如果词向量的大小为E，Transformer输出的特征大小为H，在一般的Bert中，E=H=768，从信息容量的角度来说这其实是不太合理的：E只包含了词语自身的信息，而H则是含有了上下文信息后的表征，理论上H应该远大于E。（当然这种话其实反着说也合理，有了上下文的消息消除了词向量中的一词多义性，可以用更少的编码来表示，不过论文里怎么说就怎么回事吧）因此Albert先把词语嵌入到一个比较小的词向量空间中，然后在输入Encoder以前再用一个线性层将其放大来减少一些参数量。当然于此同时更少的词向量特征数也可以增大词向量被监督的频率，更快的更新。


2. 引入句子顺序判断任务

AlBert认为，NSP任务很可能只是在促使算法学习两句话之间的主题差异而不是逻辑关系，因此AlBert将Bert中的NSP任务换成了SOP任务（Sentence Order Prediction），让模型判断两句相邻的话是否被调换了顺序。SOP任务有效的改善了模型的表现。


3. 跨层参数共享

实验发现，如果让所有的EncoderLayer的参数都进行共享，尽管大量减少了参数量，但是在事实上竟然不会损失太多的精度。于是AlBert中的EncoderLayer层无论在第几层都是共享同一套参数（其实我认为是否只共享中间几层会得到更好的效果，毕竟首尾层解码编码的是原始文本信息，需要的参数肯定还是比较大区别）通过这种方法大大减少了参数量和提高了训练速度。

![transformers_chrono](https://image.panwenbo.icu/blog20210730121119.png)

但是与之对应的训练数据并没有减少，因此需要增大Transformer在单层的参数量来使得网络依然具有和Bert差不多的参数。在最终的版本AlBert xxlarge中的hidden_size达到了4096，64头的Attention，16384的FFN隐藏层数量。速度变得更慢，但是也得到了更高的GLUE分数，在磨平了精度的劣势的同时也磨平了速度的优势。

AlBert中的一万多的FFN隐藏层单元数导致尽管只训练一层参数，AlBert训练的也比Bert慢得多，这是AlBert用一层的25倍的连接数量代替了Bert的24层TransformerEncoderLayer的结果。如果是因为不同的层之间学习的内容有一定的相似性而导致只使用一层Transformer能有接近多层网络的表现，那么Albert就是某种层面上Bert的相反面————它得益于不同层学习内容的相同之处而受损于不同层学习内容的独特性。那么我们可能会想，有没有一种办法可以根据输入数据的不同在Transformer中使用不同的参数进行计算呢？Google的Switch Transformer给出了一种可能的解决方案，不过人家不是冲着提高参数的利用效率这个角度来研究的。考虑到篇幅问题，关于Switch Transformer的内容可能要到以后再补充。

In [1]:
# 输出Albert的模型信息，可以看到Albert只有一个编码层
from transformers import AlbertConfig, AlbertModel

configuration = AlbertConfig()
albert = AlbertModel(configuration)
print(albert)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=4096, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((4096,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=4096, out_features=4096, bias=True)
              (key): Linear(in_features=4096, out_features=4096, bias=True)
              (value): Linear(in_features=4096, out_features=4096, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False

### ERNIE 2.0
ERNIE 2.0 来自百度的《ERNIE 2.0: A Continual Pre-Training Framework for Language Understanding》。是百度ERNIE的进阶版本。于此同时还有一个清华和华为合作的ERNIE，其思路和百度ERNIE还是有着比较大差异的。考虑到百度ERNIE在GLUE分数上的惊艳表现，这里就只介绍百度的ERNIE。值得一提的是，这两个ERNIE缩写的来源都很强硬（百度ERNIE：`E`nhanced `R`epresentation through k`N`owledge `I`nt`E`gration，清华ERNIE：`E`nhanced Language `R`epresentatio`N` with `I`nformative `E`ntities）

百度深谙有多少人工就有多少智能的道理，没有在BERT的结构上做出任何改动，而是一口气增加了7个新的预训练任务，并且全部都是自监督任务。通过这种方式把人类对于语言的先验知识输入到模型中，在GLUE排行榜上拿到了平均90分的高分。接下来一一介绍这7个任务：

1. **Word-aware Pre-training Tasks**

   1. **Knowledge Masking Task**

      知识遮罩的方式最早是在ERNIE 1.0中使用的，简单来说就是相比于Bert每次随机选择一些单词mask，ERNIE 1.0使用了NER算法来采样一些连续的几个mask，使得整个命名实体都被遮住需要预测，这就防止BERT仅根据短语中的某一个词语预测出Mask词语，对Bert理解语义信息提出更高要求。

      ![截屏2021-07-31 上午9.14.05](https://image.panwenbo.icu/blog20210731091420.png)

   2. **Capitalization Prediction Task**

      这个任务是一个二分类任务，要求模型判断输入词语是否被大写（如UK，New York等）。研究者发现区分大小写的模型在命名实体识别任务（NER）中表现的更好，而不区分大小写的任务在其他领域表现的更好，那么这种有意地让模型注意大小写的任务将会提高不区分大小写模型在NER等任务中的表现。

   3. **Token-Document Relation Prediction Task**

      这个任务也是个二分类任务，要求模型判断输入词语是否还会在输入词语所属的文档中再次出现。经验认为会在文档中多次出现的词语一般是与主题有关的关键词，把它们识别出来是有益的。比如“福建”，“奥运”相对于“提高”，“幸运”就更有可能在同一主题的文本中多次出现。

2. **Structure-aware Pre-training Tasks**

   1. **Sentence Reordering Task**

      在这个任务中，输入模型的段落中的m个句子会被随机打乱，这种打乱有$k = \sum_{n=1}^m n!$种可能，模型因此需要执行一个k分类任务来正确的找到文档是如何被打乱的。这个任务可以看作是强化版的SOP任务，有助于模型理解不同句子之间的联系。我个人比较好奇k在不同的输入中是不同的，算法要如何解决这个问题。

   2. **Sentence Distance Task**

      这个任务要求模型针对输入的两句话做出分类0，1或2，0代表这两句话是相邻的，1代表这两句话是不相邻但是在同一段文本中，2代表这两句话不在同一段文本中。

3. **Semantic-aware Pre-training Tasks**

   1. **Discourse Relation Task**

      谈话关系任务的数据集来自《Mining Discourse Markers for Unsupervised Sentence Representation Learning》论文，这个任务认为，谈话中的中间词是很重要的比如在一句话*I live in Paris. But I’m often abroad.*中，But对于语义信息有着重要的作用，因此这个数据集把语段整理成形如下图的训练数据：

      | s1     | Paul Prudhomme’s Louisiana Kitchen created a sensation when it was published in 1984 |
      | ------ | ------------------------------------------------------------ |
      | **c**  | **happily,**                                                 |
      | **s2** | **This family collective cookbook is just as good**          |

      模型需要做的就是输入两句话s1&s2，预测c处的词语happily，这样地位的词语在训练集中总共统计了174个，因此这是个174分类问题。

   2. **IR Relevance Task**

      信息提取数据集包含了大量的（用户在搜索引擎输入框里输入的内容，搜索引擎跳出的条目）数据对，模型根据输入的两条信息，要判断这属于（搜索信息，用户点击的条目），（搜索信息，搜索引擎提供但是用户没点击条目），（搜索信息，无关的条目）中的哪一种，这考验了模型对于文本相关程度的认知。

这么多的任务并不是一次性全部输入模型训练，而是慢慢增多，为每个任务设置了N次的使用次数上限，一个算法把这些任务均匀的有序的提供给模型，使得模型能够不断学习新的任务又不至于灾难性遗忘。

<img src="https://image.panwenbo.icu/blog20210731100618.png" alt="截屏2021-07-31 上午10.04.05" style="zoom:50%;" />

大力出奇迹，有了这么多的任务，尽管训练数据和模型规模都没什么变化，但是ERNIE的表现还是非常惊艳的。

<img src="https://image.panwenbo.icu/blog20210731100750.png" alt="截屏2021-07-31 上午10.07.06" style="zoom:50%;" />


### GPT-1/2/3模型
GPT（Gernerative Pre-Traning)模型出现的时间比Bert要早，相比于Bert使用Transformer中的Encoder做MLM，GPT则是取用了Transformer中的Decoder部分，当然由于只有Decoder部分，GPT的Transformer的一个层只含有自注意力层而和没有来自Encoder的K和V的注意力层。实际上把GPT称为是有注意力遮罩的TransformerEncoder可能更贴切一些。GPT预测文本中的所有词语来训练一个语言模型，它每次预测第t个词语时，都只会利用到前t-1个词语的信息，这和Transformer中的Decoder是一致的。

GPT-1/2/3都是基本上相同的结构，只是在模型参数和训练数据大小上有区别。GPT-2稍微修改了GPT的结构，而GPT-3的结构和GPT-2相同，但是引入了元学习的学习方式。它们的结构参数的进步是相当大的：
| 名词  | 时间   | 训练数据 | 参数量 | 层数 | Hidden_size |
| ----- | ------ | -------- | ------ | ---- | ----------- |
| GPT   | 2018/6 | 5GB      | 1.17亿 | 12   | 768         |
| GPT-2 | 2019/2 | 40GB     | 15亿   | ≈36  | ≈1300       |
| GPT-3 | 2020/5 | 45TB     | 1750亿 | 96   | 12888       |



值得注意的是这三篇文章的标题：

- 《Improving Language Understanding by Generative Pre-Training》
- 《Language Models are Unsupervised Multitask Learners》
- 《Language Models are Few-Shot Learners》

展示一下试出来的GPT-2虚无主义回答：输入：`Here is the core of philosophy: `输出：`Here is the core of philosophy: ____. So there it is. `

In [7]:
from transformers import pipeline

generator = pipeline("text-generation", "distilgpt2")
text = generator("Here is the core of philosophy: ")[0]['generated_text']
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Here is the core of philosophy: _____________________________________(S)
There is no dogma to refute or refute arguments that are held against the fundamental notion that there is an objective reality (for example, that there is something beyond what empirical evidence is possible


### CTRL
CTRL模型来自论文《CTRL: A CONDITIONAL TRANSFORMER LANGUAGE MODEL FOR CONTROLLABLE GENERATION》，从模型规模上来讲是一个比GPT-2略大一些，使用的训练数据略多一些的一个TransformerDecoder，使用的具体结构和GPT是一模一样的。但是与之不同的是，CTRL专注于生成文本，CTRL在输入文本训练时在文本开头插入了控制代码来告诉模型这段文本的主题，这样在模型的测试阶段，也可以通过改变文本开头的控制代码来定义输出文本的主题。下面展示了即使输入文本开头是一样的，更改控制代码也可以产生完全不同的输出结果：

![截屏2021-07-31 下午4.35.22](https://image.panwenbo.icu/blog20210731163533.png)

那么在训练的时候，怎么知道要给输入文本贴上什么控制代码呢？论文根据数据的来源提取了一些关键词作为控制代码：从Reddit的健身板块获取的文本就被标以`r/fitness`的控制代码；从“一个人才能看”板块获取的文本就有`r/childfree`的控制代码；从维基百科获取的文本就被归入`Wikipedia`；对于从OpenWebText获取的数据，文本所属的网址就会被插入到文本开头作为控制代码……通过这种方式训练出来的模型有以下有趣之处：

可以通过修改输入prompt开头的网址内容来使模型编一个指定日期，事件的新闻。同样是美国总统会见英国人的新闻，指定不同的日期，正文里的美国总统就不是同一个人了。

![截屏2021-07-31 下午4.35.22](https://image.panwenbo.icu/blog20210731164602.png)

类似翻译/回答这样的控制代码如果被输入，那么后面的模型的输出就会是为了完成这个任务而生成的。

![截屏2021-07-31 下午4.35.22](https://image.panwenbo.icu/blog20210731165031.png)

可以混合两种无关的控制代码来得到同时处于两种主题下的生成文本。下面展示的是指定评分的评价，Rating和Review是不同的控制代码

![截屏2021-07-31 下午4.35.22](https://image.panwenbo.icu/blog20210731165421.png)

有控制代码的文本生成可以被视为是一种条件概率，那么只要我们运用贝叶斯公式：

$$
P(c|x) \propto P(x|c)P(c) 
$$

就可以根据文本计算出它可能所属的领域：

![截屏2021-07-31 下午4.35.22](https://image.panwenbo.icu/blog20210731165923.png)

大力出奇迹，这个模型还是很有趣的。

### XLNet
XLNet是正统的自回归语言模型，它是一个TransformerDecoder，也就是说它依赖于公式：
$$
\argmax_{theta} P(x_T, x_{T-1}, \cdots, x_1; \theta) = \argmax_{theta} \prod_{t=1}^T P(x_t| x_{<t}; \theta） \\
$$
GPT一家也是使用这个公式的，但是因为使用这种方法设计语言模型，模型在预测第t个词语的时候，只能参考到前t-1个词语，无法利用双向信息使得自回归模型在和Bert的较量中落了下风。于是XLNet转变思路——传统模型按照从左往右的顺序处理序列，XLNet则按照一个随机的全排列顺序计算条件概率。也就是如果GPT按照如下顺序计算语句概率：
$$
P(x_{4,3,2,1})=P(x_1)P(x_2|x_1)P(x_3|x_{2,1})P(x_4|x_{3,2,1})
$$
XLNet按照的顺序就有可能是:
$$
P(x_{2,3,1,4})=P(x_4) P(x_1|x_4) P(x_3|x_{1,4}) P(x_2|x_{3,1,4})
$$
XLNet一样可以计算出在2314这个全排序下词语序列的概率，同时也使得预测词语时能够访问到后面的词语，而想要计算$P(x_2|x_{3,1,4})$之类的条件概率不需要真的修改序列，只需要合理的调整注意力Mask即可。这就是XLNet的核心思想，当然还需要进行一些其他改动才能使这种方法成为可行，这里由于篇幅原因无法介绍太多。

XLNet的另一优势在于它使用了Transformer XL作为它的结构。总而言之依靠这两项改进XLNet在GLUE数据集表现上也远远超过了Bert。

### BART\UniLM\T5
这几个整体来说的思路都是差不多的，它们既不属于Bert一类的自编码模型，也不是GPT一类的自回归模型，它们是使用一整个的TransformerEncoder-Decoder结构来进行语言模型的训练，由于Encoder和Decoder不需要有相同的输出长度，我们在要求Decoder输出完整文档的同时可以对输入Encoder进行任意的损坏，包括：

1. 类似spanBert的Mask连续几个字符的操作，这里无论Mask多少个字符，在Encoder眼里都只有一个Mask。
2. 直接删除几个词语。
3. 打乱文档中的句子顺序。

事实证明，这些Encoder-Decoder的预训练模型非常适合用于生成摘要，翻译等微调任务。


In [44]:
from transformers import BartTokenizerFast, BartForConditionalGeneration

text = """The tower is 324 metres (1,063 ft) tall, about the same height as an 
81-storey building, and the tallest structure in Paris. Its base is square, 
measuring 125 metres (410 ft) on each side. During its construction, the Eiffel 
Tower surpassed the Washington Monument to become the tallest man-made structure 
in the world, a title it held for 41 years until the Chrysler Building in New York 
City was finished in 1930. It was the first structure to reach a height of 300 metres. 
Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is 
now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, 
the Eiffel Tower is the second tallest free-standing structure in France after the 
Millau Viaduct."""
tokenizer = BartTokenizerFast.from_pretrained("sshleifer/distilbart-cnn-12-6")
summarization = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")

inputs = tokenizer([text], return_tensors='pt')
summary_ids = summarization.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

[' The Eiffel Tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building . It is the tallest structure in Paris and is the second tallest free-standing structure in France after the Millau Viaduct .']
